# Fase PreDB

El objetivo es preparar las tablas para nuestra Base de Datos
Vamos a tener las siguientes tablas y en el siguiente orden:
- fechas
- combustibles
- categorías
- unidades
- historico_precios

In [1]:
# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipular estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y arrays multidimensionales
import datetime                 # Para manejar fechas y horas

import os
import sys
sys.path.append("../")
from src.SupportPreDB import (
    crear_dictio,
    df_a_db
)

In [2]:
df_gas = pd.read_csv("../datos/01_PreciosGas/03_TablaPreciosGasFinal.csv",index_col="Unnamed: 0")
df_luz = pd.read_csv("../datos/02_PreciosLuz/02_PreciosLuzFinal.csv",index_col="Unnamed: 0")
df_gasolina = pd.read_csv("../datos/03_PreciosCombustible/01_TablaPreciosCombustible.csv",index_col="Unnamed: 0")
df_agua = pd.read_csv("../datos/04_PreciosAgua/01_PreciosAguaFinal.csv",index_col="Unnamed: 0")

# Tabla Fechas
Para esta tabla cogemos los valores únicos de las fechas de cualquiera de nuestras tablas

In [3]:
df_fechas = pd.DataFrame(df_gas["fecha"].unique())
df_fechas.index = df_fechas.index + 1
df_fechas.reset_index(inplace=True)
df_fechas = df_fechas.rename(columns={0:"fecha"})
df_fechas.sample()

,index,fecha
29,30,2021-06-01


In [4]:
df_fechas.to_csv("../datos/05_TablasPreDB/01_tabla_fechas.csv")

# Tabla combustibles
Para esta tabla sacamos los valores únicos del tipo combustible de la tabla "Combustibles"

In [5]:
df_combustibles = pd.DataFrame(df_gasolina["combustible"].unique())
df_combustibles = df_combustibles.rename(columns={0:"combustible"})
df_combustibles = pd.concat([pd.DataFrame({"index": [0], "combustible": ["N/A"]}), df_combustibles], ignore_index=True)
df_combustibles = df_combustibles.drop(columns="index")
df_combustibles.index = df_combustibles.index + 1
df_combustibles.reset_index(inplace=True)
df_combustibles

,index,combustible
0,1,N/A
1,2,Gasolina 95
2,3,Gasolina 98
3,4,Diésel
4,5,Diésel Premium


In [6]:
df_combustibles.to_csv("../datos/05_TablasPreDB/02_tabla_combustibles.csv")

# Tabla categorías
Categorías de los valores que estamos comprobando:
- luz
- gas
- combustible
- agua

In [7]:
categorias = ["luz","gas","combustible","agua"]
df_categorias = pd.DataFrame(categorias)
df_categorias.index = df_categorias.index + 1
df_categorias.reset_index(inplace=True)
df_categorias = df_categorias.rename(columns={0:"categoria"})
df_categorias

,index,categoria
0,1,luz
1,2,gas
2,3,combustible
3,4,agua


In [8]:
df_categorias.to_csv("../datos/05_TablasPreDB/03_tabla_categorias.csv")

# Tabla unidades
cada Precio tiene su unidad que son:
- €/L para la gasolina
- €/m3 para el agua
- €/MWh para la luz y el Gas

In [9]:
unidades = ["€/L","€/m3","€/MWh"]
df_unidades = pd.DataFrame(unidades)
df_unidades.index = df_unidades.index + 1
df_unidades.reset_index(inplace=True)
df_unidades = df_unidades.rename(columns={0:"unidad"})
df_unidades

,index,unidad
0,1,€/L
1,2,€/m3
2,3,€/MWh


In [10]:
df_unidades.to_csv("../datos/05_TablasPreDB/04_tabla_unidades.csv")

# Tabla Histórico_precios
Para esta tabla haremos un concat de todas las tablas que tenemos, primero las prepararemos para unirlas y después reemplazaremos los valores por ids donde sea necesario

La tabla va a quedar con estas columnas (en todas):
- categoria : Si es luz, gas, combustible o agua
- combustible: en caso de ser combustible si es 95,98, diesel o diesel premium
- precio: el coste en €
- unidad: la unidad del coste en € (si es por litro, metro cúbico o Megavatio hora)
- fecha: La fecha (en este caso el mes)

### Tabla Histórico_luz


In [11]:
df_luz.sample()

,index,precio,fecha
37,38,205.54,2022-02-01


In [12]:
df_luz = df_luz.drop(columns="index")
df_luz["categoria"] = "luz"
df_luz["combustible"] = "N/A"
df_luz["unidad"] = "€/MWh"
df_luz = df_luz[["categoria","combustible","precio","unidad","fecha"]]
df_luz.sample()

,categoria,combustible,precio,unidad,fecha
19,luz,N/A,41.0,€/MWh,2020-08-01


### Tabla Histórico_gas

In [13]:
df_gas.sample()

,index,precio_gas,fecha
19,20,7.23,2020-08-01


In [14]:
df_gas = df_gas.drop(columns="index")
df_gas["categoria"] = "gas"
df_gas["combustible"] = "N/A"
df_gas["unidad"] = "€/MWh"
df_gas = df_gas.rename(columns={"precio_gas":"precio"})
df_gas = df_gas[["categoria","combustible","precio","unidad","fecha"]]
df_gas.sample()

,categoria,combustible,precio,unidad,fecha
13,gas,N/A,10.4,€/MWh,2020-02-01


### Tabla Histórico_combustible

In [15]:
df_gasolina.sample()

,Fecha,Precio,combustible
81,2020-10-01,1.069652,Diésel


In [ ]:
df_gasolina["categoria"] = "combustible"
df_gasolina["unidad"] = "€/L"
df_gasolina = df_gasolina.rename(columns={"Fecha":"fecha",
                                          "Precio":"precio"})
df_gasolina = df_gasolina[["categoria","combustible","precio","unidad","fecha"]]

### Tabla Histórico_agua